# Creating your own EmissionModel

To create your own `EmissionModel` all you need to do is define each indiviudal emission type and relate them. This operation will define a tree-like structure where each model links to at least one other model. 

In the sections below we will detail how to define each different type of emission operation but there are a few arguments needed by an ``EmissionModel` regardless of the operation:

- A **label** which identifies the model. These must be unique within a single emission model's "tree".
- An emitter to act on. This is given by passing a string to the emitter argument. Currently the possible emitters are: ``"stellar"``, ``"blackhole"``, and ``"galaxy"``.


In [ ]:
from synthesizer.dust.attenuation import PowerLaw
from synthesizer.dust.emission import Blackbody
from synthesizer.emission_models import EmissionModel
from synthesizer.grid import Grid
from unyt import cm, dimensionless, kelvin, km, s

# Get the grid which we'll need for extraction
grid_dir = "../../../tests/test_grid"
grid_name = "test_grid"
grid = Grid(grid_name, grid_dir=grid_dir)


## Defining an extraction

To define an extraction we simply need to pass a `Grid` to extract from and a key to extract (with the option of providing an escape fraction).

In [ ]:
transmitted = EmissionModel(
    "transmitted", grid=grid, extract="transmitted", fesc=0.3
)
print(transmitted)

## Defining a combination

To define a combination we simply pass the models we want combined to the `combine` keyword (along with a label).

In [ ]:
# Define models to combine
linecont = EmissionModel(
    "linecont",
    grid=grid,
    extract="linecont",
    mask_attr="log10ages",
    mask_thresh=7 * dimensionless,
    mask_op="<",
    fesc=0.7,
)
nebular_cont = EmissionModel(
    "nebular_continuum",
    grid=grid,
    extract="nebular_continuum",
    mask_attr="log10ages",
    mask_thresh=7 * dimensionless,
    mask_op="<",
)

# Define the combined model
nebular = EmissionModel("nebular", combine=(linecont, nebular_cont))
print(nebular)

## Defining an attenuation

To define an attenuated emission model we need a dust curve, the model to apply the dust to, and an optical depth (once again along with a label).

Any property can also have strings passed instead of numbers. When a string is passed the spectra generator method will look for an attribute on an emitter using this string. This is only applicable for certain properties, e.g. `tau_v` and `fesc`.

In [ ]:
attenuated = EmissionModel(
    "attenuated",
    dust_curve=PowerLaw(slope=-1),
    apply_dust_to=nebular,
    tau_v="tau_v",
)
print(attenuated)

## Defining a generation

To define a generation model we only need to provide the generator class (i.e. a ``Blackbody``).

In [ ]:
from synthesizer.dust.emission import Blackbody

bb_emission = EmissionModel(
    "blackbody_emission",
    generator=Blackbody(50 * kelvin),
)
print(bb_emission)

If we want to scale the generated emission by another emission model we can pass that as the ``lum_intrinsic_model``.

In [ ]:
scaled_bb_emission = EmissionModel(
    "blackbody_emission",
    generator=Blackbody(50 * kelvin),
    lum_intrinsic_model=transmitted,
)
print(scaled_bb_emission)

For the special case of energy balance dust emission you can pass both the intrinsic and attenuated models to scale by the emission reprocessed by dust.

In [ ]:
dust_emission = EmissionModel(
    "blackbody_emission",
    generator=Blackbody(50 * kelvin),
    lum_intrinsic_model=transmitted,
    lum_attenuated_model=attenuated,
)
print(dust_emission)

## Including a mask

A mask can be included in any step by passing `mask_attr`, `mask_thresh`, and `mask_op`.

In [ ]:
masked_transmitted = EmissionModel(
    "masked_transmitted",
    grid=grid,
    extract="transmitted",
    fesc="fesc",
    mask_attr="log10ages",
    mask_thresh=7 * dimensionless,
    mask_op="<",
)

But we aren't tied to having only a single mask on a step. If we want to add more masks we can use `add_mask` as we did on the premade models, these will be combined with an `and` at the point of spectra generation.

In [ ]:
masked_transmitted.add_mask("metallicities", "<", 0.01 * dimensionless)
masked_transmitted.add_mask("log10ages", "<", 6 * dimensionless)
print(masked_transmitted)

## Including related models

In many instances you'll have a main emission you care about plus some extra emission models that get produced along the way but do not feature in the main "tree". To include these models and ensure their spectra is generated by calls to ``get_spectra`` or ``get_lines`` methods you can pass them to the ``related_models`` argument at instantiation.

In [ ]:
# Define the combined model
nebular = EmissionModel(
    "nebular", combine=(linecont, nebular_cont), related_models=(transmitted)
)
print(nebular)

## Fixing parameters

When an emission model is used in a ``get_spectra`` method any parameters the model does not define will be extracted from the emitter. If you would like to give any of the values fixed values you can pass a dictionary of parameters to a model at instantiation. For demonstration purposes we'll create an AGN NLR emission model with fixed NLR parameters. This can also be done with a setter after instantiation (see the model modification [docs](modify_models.ipynb)).

In [ ]:
# Get the NLR and BLR grids
nlr_grid = Grid("test_grid_agn-nlr", grid_dir="../../../tests/test_grid")

nlr = EmissionModel(
    label="nlr",
    extract="nebular",
    grid=nlr_grid,
    fesc=0.1,  # covering fraction
    fixed_parameters={
        "ionisation_parameter_nlr": 0.01,
        "hydrogen_density_nlr": 1e4 / cm**3,
        "covering_fraction_nlr": 0.1,
        "velocity_dispersion_nlr": 500 * km / s,
    },
)
print(nlr)

## Using post processing functions

You may want to apply some post processing to your emission that isn't offered by the Synthesizer machinery. In these cases you can pass a list of "post processing functions" to the ``EmissionModel``. These functions will be called in the order they are passed after all spectra have been generated. 

These post processing functions must take:

- A dictionary of "emissions" (spectra/lines). Where the dictionary will be filled by key-value pairs of emission model labels and the emission they produce (``Sed``/``LineCollection``) ``{<model.label>: <emission>}``. 
- A dictionary of emitters. Where the dictionary will be ``{"stellar": Stars, "blackhole": BlackHoles, "galaxy": Galaxy}``.
- An ``EmissionModel``. 

When ``get_spectra`` is called and all spectra have been generated any post processing functions will be called with these arguments (in the order listed above).

Below we define a **fake** function to perform some arbitrary mass scaling of the spectra. 

In [ ]:
def scale_by_mass(emission, emitters, model):
    """
    Scale the emission by mass.

    Args:
        emission (dict): A dictionary of emissions.
        emitters (dict): A dictionary of emitters.
        model (EmissionModel): The model defining the emissions.
    """
    # Loop over emissions
    for key, value in emission.values():
        # Get model
        this_model = model[key]

        # Get the emitter
        emitter = emitters[this_model.emitter]

        # Scale the emission
        value *= emitter.masses

    return emission


nebular = EmissionModel(
    "nebular",
    combine=(linecont, nebular_cont),
    related_models=(transmitted),
    post_processing=(scale_by_mass,),
)

## Putting it all together

In the code below we'll reconstruct the `TotalEmission` model with the Charlot&Fall+2000 like attenuation operation explicitly to demonstrate constructing a complex model.

In [ ]:
# Define the extractions
transmitted = EmissionModel(
    "transmitted",
    grid=grid,
    extract="transmitted",
    fesc=0.3,
)
incident = EmissionModel("incident", grid=grid, extract="incident", fesc=0.0)
escaped = EmissionModel("escaped", grid=grid, extract="transmitted", fesc=0.7)
linecont = EmissionModel(
    "linecont",
    grid=grid,
    extract="linecont",
    mask_attr="log10ages",
    mask_thresh=7 * dimensionless,
    mask_op="<",
    fesc=0.7,
)
nebular_cont = EmissionModel(
    "nebular_continuum",
    grid=grid,
    extract="nebular_continuum",
    mask_attr="log10ages",
    mask_thresh=7 * dimensionless,
    mask_op="<",
)

# Combine the extractions
nebular = EmissionModel("nebular", combine=(linecont, nebular_cont))
reprocessed = EmissionModel("reprocessed", combine=(nebular, transmitted))

# Apply the young and old dust attenuation
young_attenuated = EmissionModel(
    "young_attenuated",
    dust_curve=PowerLaw,
    apply_dust_to=reprocessed,
    tau_v=("tau_v", 0.67),
    mask_attr="log10ages",
    mask_thresh=7 * dimensionless,
    mask_op="<",
)
old_attenuated = EmissionModel(
    "old_attenuated",
    dust_curve=PowerLaw,
    apply_dust_to=reprocessed,
    tau_v="tau_v",
    mask_attr="log10ages",
    mask_thresh=7 * dimensionless,
    mask_op=">=",
)

# And combine them into a single attenuated spectra
attenuated = EmissionModel(
    "attenuated", combine=(young_attenuated, old_attenuated)
)
emergent = EmissionModel("emergent", combine=(attenuated, escaped))

# Create a dust emission model
dust_emission = EmissionModel(
    "dust_emission",
    generator=Blackbody(temperature=100 * kelvin),
    lum_intrinsic_model=incident,
    lum_attenuated_model=emergent["attenuated"],
)

# And bring everything together into the total emission
total = EmissionModel("total", combine=(emergent, dust_emission))

total.plot_emission_tree()
print(total)